In [100]:
import os
import pandas as pd
import numpy as np
import time

In [101]:
def make_TFNP_tools(df_gt, df_pred):    
    df_gt = df_gt.drop(labels=['Frame_No.'], axis = 1)
    if 'PC_All' in df_gt.keys():
        df_gt = df_gt.drop(labels=['PC_All'], axis = 1)
    
    dict_TFNP = {} 
    for column in df_gt.columns:
        if column != 'phase':   
            if column in df_pred.columns:
                for a in range(len(df_gt[column])):
                    if column not in dict_TFNP:
                        dict_TFNP[column] = {'TP' : 0, 'TN' : 0, 'FP' : 0, 'FN' : 0}

                        if df_gt[column][a] == 1 and  df_pred[column][a] == 1:
                            dict_TFNP[column]['TP'] = dict_TFNP[column]['TP'] + 1

                        elif df_gt[column][a] == 1 and df_pred[column][a] == 0:
                            dict_TFNP[column]['FN'] = dict_TFNP[column]['FN'] + 1

                        elif df_gt[column][a] == 0 and df_pred[column][a] == 1:
                            dict_TFNP[column]['FP'] = dict_TFNP[column]['FP'] + 1

                        elif df_gt[column][a] == 0 and df_pred[column][a] == 0:
                            dict_TFNP[column]['TN'] = dict_TFNP[column]['TN'] + 1

                    else:
                        if df_gt[column][a] == 1 and  df_pred[column][a] == 1:
                            dict_TFNP[column]['TP'] = dict_TFNP[column]['TP'] + 1

                        elif df_gt[column][a] == 1 and df_pred[column][a] == 0:
                            dict_TFNP[column]['FN'] = dict_TFNP[column]['FN'] + 1

                        elif df_gt[column][a] == 0 and df_pred[column][a] == 1:
                            dict_TFNP[column]['FP'] = dict_TFNP[column]['FP'] + 1

                        elif df_gt[column][a] == 0 and df_pred[column][a] == 0:
                            dict_TFNP[column]['TN'] = dict_TFNP[column]['TN'] + 1

    return dict_TFNP

In [102]:
def make_tool_matrix(df_gt, df_pred, gt):
    dict_TFNP_tools = make_TFNP_tools(df_gt, df_pred)
    
    list_precision = []
    list_acc = []
    list_recall = []
    df_temp_tools = pd.DataFrame(columns = ['tools', 'acc', 'precision','recall'])
    for column in dict_TFNP_tools:
        print(column, dict_TFNP_tools[column])
        acc = round((dict_TFNP_tools[column]['TP']+dict_TFNP_tools[column]['TN'])/(dict_TFNP_tools[column]['TP']+dict_TFNP_tools[column]['FP']+dict_TFNP_tools[column]['TN']+dict_TFNP_tools[column]['FN']),2)
        if (dict_TFNP_tools[column]['TP']+dict_TFNP_tools[column]['FP']) != 0:
            precision = round((dict_TFNP_tools[column]['TP'])/(dict_TFNP_tools[column]['TP']+dict_TFNP_tools[column]['FP']),2)
        else:
            precision = ''
        if (dict_TFNP_tools[column]['TP']+dict_TFNP_tools[column]['FN']) != 0:
            recall = round((dict_TFNP_tools[column]['TP'])/(dict_TFNP_tools[column]['TP']+dict_TFNP_tools[column]['FN']),2)
        else:
            recall = ''
        
        print([column, acc, precision, recall])
        TP = dict_TFNP_tools[column]['TP']
        TN = dict_TFNP_tools[column]['TN']
        FP = dict_TFNP_tools[column]['FP']
        FN = dict_TFNP_tools[column]['FN']
        
        df_column = pd.DataFrame([[column, TP, TN, FP, FN, acc, precision, recall]], columns = ['tools','TP','TN','FP','FN', 'acc', 'precision','recall'])
        df_temp_tools = df_temp_tools.append(df_column)

    return df_temp_tools

In [ ]:
path_root = r'' # gt, pred 폴더를 담은 상위 폴더
path_gt_folder = os.path.join(path_root, 'gt')
list_path_pred_folder = [path_ for path_ in os.listdir(path_root) if path_[:len('pred_winsize')] == 'pred_winsize']
list_gt = os.listdir(path_gt_folder)

df_compare = pd.DataFrame()
df_compare_tools = pd.DataFrame()
for gt in list_gt:
    print(gt)
    path_gt = os.path.join(path_gt_folder, gt)
    df_gt = pd.read_csv(path_gt, encoding='cp949')
    for pred_folder in list_path_pred_folder:
        path_pred_folder = os.path.join(path_root, pred_folder)
        list_pred_file = os.listdir(path_pred_folder)
        for pred_file in list_pred_file:
            path_pred_file = os.path.join(path_pred_folder, pred_file)
            if pred_file[:24] == gt[:24]:
                df_pred = pd.read_csv(path_pred_file, encoding='cp949')
                df_temp_tools = make_tool_matrix(df_gt, df_pred, gt)
                df_temp_tools.insert(0, 'winsize',[int(pred_folder[len('pred_winsize'):])]*len(df_temp_tools))
                df_temp_tools.insert(0, 'video',[gt]*len(df_temp_tools))

                df_compare_tools = df_compare_tools.append(df_temp_tools)

In [ ]:
df_compare_tools_origin = df_compare_tools
df_compare_tools_sorted = df_compare_tools.sort_values(['video', 'tools','winsize'],ascending = [True, True, True])
df_compare_tools_sorted

In [106]:
# 도구 전체 테이블 저장
# 컬럼 : video, winsize, tools, acc, precision, recall, TP, TN, FP, FN
path = r''
name = r''
path_save_excel = os.path.join(path, name)
df_compare_tools_sorted.to_excel(path_save_excel)

In [ ]:
# 도구 종합 metrix 테이블 제작
df_tool_totalMetrix = pd.DataFrame()
for winsize in df_compare_tools_origin['winsize'].unique():
    for tool in df_compare_tools_origin['tools'].unique():
        df_winsize_tool = df_compare_tools_origin[(df_compare_tools_origin['winsize'] == winsize) & (df_compare_tools_origin['tools'] == tool)]
        print(df_winsize_tool)
        TP_total = df_winsize_tool['TP'].sum()
        TN_total = df_winsize_tool['TN'].sum()
        FP_total = df_winsize_tool['FP'].sum()
        FN_total = df_winsize_tool['FN'].sum()
        
        acc = round((TP_total+TN_total)/(TP_total+FP_total+TN_total+FN_total),2)
        if (TP_total+FP_total) != 0:
            precision = round((TP_total)/(TP_total+FP_total),2)
        else:
            precision = ''
        if (TP_total+FN_total) != 0:
            recall = round((TP_total)/(TP_total+FN_total),2)
        else:
            recall = ''
        
        df_winsize_tool_tmp = pd.DataFrame([[winsize, tool, TP_total, TN_total, FP_total, FN_total, acc, precision, recall]], columns = ['winsize','tools','TP','TN','FP','FN', 'acc', 'precision','recall'])

        df_tool_totalMetrix = df_tool_totalMetrix.append(df_winsize_tool_tmp)

In [109]:
df_tool_totalMetrix_bywinsize = df_tool_totalMetrix.sort_values(['winsize'],ascending = [True])
df_tool_totalMetrix_bytools = df_tool_totalMetrix.sort_values(['tools', 'winsize'],ascending = [True, True])

In [110]:
# 도구 종합 metrix 테이블 저장
# 컬럼 : winsize, tools, acc, precision, recall, TP, TN, FP, FN
path = r''
dataset_name = ''
name_bytools = rf'{dataset_name}_bytools.xlsx'
name_bywinsize = rf'{dataset_name}_bywinsize.xlsx'
path_save_excel_bywinsize = os.path.join(path, name_bywinsize)
path_save_excel_bytools = os.path.join(path, name_bytools)
df_tool_totalMetrix_bywinsize.to_excel(path_save_excel_bywinsize)
df_tool_totalMetrix_bytools.to_excel(path_save_excel_bytools)